# Training Resolution model
##### Tested on kernel `TensorFow on GPU` as defined in the image `landerlini/lhcbaf:v0p8`

This notebook is part of a pipeline. It requires the preprocessing step defined in the 
[GAN preprocessing notebook](./Preprocessing-GANs.ipynb) and the trained model is 
validated in the [Resolution-validation](./Resolution-validation.ipynb) notebook.

### Environment and libraries
As for the other trainings, we are handling the GPU with TensorFlow.
To make sure the GPU is found, we print below the system name of the accelerator.

In [1]:
import numpy as np 
import pandas as pd 
import dask.dataframe as ddf
import os
from os import environ
import time

## Remove annoying warnings 
environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

print ("GPU:", tf.test.gpu_device_name())
print ("Running Tensorflow", tf.version.VERSION, "Keras", tf.keras.version())


E0000 00:00:1752851189.262648  641245 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752851189.305015  641245 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752851189.717267  641245 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752851189.717298  641245 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752851189.717301  641245 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752851189.717302  641245 computation_placer.cc:177] computation placer already registered. Please check linka

GPU: 
Running Tensorflow 2.19.0 Keras 3.8.0


## Loading the data

The data are loaded with our custom `FeatherReader` helper class, defined in the local module [`feather_io`](./feather_io.py).

In this notebook, we are using:
 * *training data*: to optimize the weights of the network
 * *validation data*: to evaluate overtraining effects

In [2]:
from feather_io import FeatherReader    

train_dataset = FeatherReader(environ.get("TRAIN_DATA", "/tmp/resolution-train")).as_tf_dataset()
validation_dataset = FeatherReader(environ.get("VALIDATION_DATA", "/tmp/resolution-validation")).as_tf_dataset()

A chunk of data is loaded to ease the construction of the model, for example defining the shapes of the input and output tensors.

In [3]:
X, y = next(iter(train_dataset.batch(1_000_000)))
y.shape

TensorShape([353186, 9])

## Definition of the model
The GANs used in this module are composed of three different neural networks trained simultaneously, namely:
 * a *generator* neural network that takes as an input the *condition* (such as the generator-level features) and the *random noise* and formulate predictions for the output
 * a *discriminator* neural network trained to identify whether a sample was part of the reference dataset or was produced by the generator
 * a *referee* network that mimics the configuration of the discriminator, is trained with a much larger learning rate.
 
Generator and discrminator are part of the architecture of GANs since [they were proposed in 2014](https://arxiv.org/abs/1406.2661),
with the underlying idea that the generator has to be trained to worsen the classification ability of the discriminator.
Generator and discriminator are players of a game in which they try to drive the classification loss function towards
higher and lower values, respectively. The outcome of the game that we wish to reach an equilibrium between 
the two players (Nash equilibrium) that forces both players to improve their abilities along the training 
and results into a generator providing samples with distributions almost indistinguishable from those of the 
reference sample.

Unfortunately, using the loss function of the discriminator as adversarial objective of two neural networks 
makes it harder to interpret it as metric for the goodness of the trained model.
A high value of the loss may be obtained with both an excellent generator evaluated by an excellent discriminator, 
or by an awful generator evaluated by an awful discriminator.
To evaluate the progress of the training and assess the overall quality of the generator, we introduced a third
neural network to the game that does not contribute to the training of the generator, but is a spectator providing 
and independent assessment of the agreement between the generated and the trained sample. We call this the *referee
network*. Since it does not provide a feedback to the generator, larger jumps in the parameter space do not cause
the training procedure of the generator to derail because of confusing information. Therefore, the learning rate can be larger.
A large learning rate is also useful to provide timely updates on the status of the network, identifying quickly 
the discrepancies between the generated and the reference sample.

The referee network was found to provide useful information when comparing the loss on the 
training and validation datasets. Indeed, we observed the ability of the *discriminator* 
to learn "by heart" some of the outlayers in the reference sample, resulting in a classification
that does not generalize to the validation dataset.

As for other tasks, we observed that L2 regularization is useful to contrast overtraining.
In addition, L2 regularization is expected to contrast sharp boundaries in the classification
function that may lead to explosions of the gradient when training the generator.

As discussed for the acceptance models, we adopted very deep neural networks with skip connections to 
achieve decent results with limited effort in hyperparameter optimization. This choice might be reviewed
with additional effort on tuning.

### Generator architecture
The generator takes the input conditions and the random noise as inputs and provides the target features as an output.
In formula,
$$
g: \mathbb R^{N_I} \otimes \mathbb R^{N_R} \to \mathbb R^{N_O} 
$$
where $N_I$ and $N_O$ respresent the number of input and output features, respectively; while $N_R$ is the 
dimension of the random noise

The activation function of the last layer is linear to avoid any constraint on the generated features.


In [4]:
g_input = tf.keras.layers.Input(shape=[X.shape[1]])
random = tf.keras.layers.Input(shape=[128])

g_dense_cfg=dict(units=128, activation='tanh', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.L2(1e-3))

x = tf.keras.layers.Concatenate(axis=1)((g_input, random))
x = tf.keras.layers.Dense(**g_dense_cfg)(x)
for i in range(10):
    x_ = tf.keras.layers.Dense(**g_dense_cfg)(x)
    x = x + x_

ny = y.shape[1]
output = tf.keras.layers.Dense(ny)(x) #+ random[:, :ny]
    
    
generator = tf.keras.Model(inputs=[g_input, random], outputs = output)
generator.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 12)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 140)       │          0 │ input_layer[0][0… │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     18,048 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     16,512 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128)       │          0 │ dense[0][0],      │
│                     │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128)       │     16,512 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 128)       │          0 │ add[0][0],        │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     16,512 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 128)       │          0 │ add_1[0][0],      │
│                     │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │     16,512 │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 128)       │          0 │ add_2[0][0],      │
│                     │                   │            │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     16,512 │ add_3[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 128)       │          0 │ add_3[0][0],      │
│                     │                   │            │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     16,512 │ add_4[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 128)       │          0 │ add_4[0][0],      │
│                     │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 128)       │     16,512 │ add_5[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 128)       │          0 │ add_5[0][0],      │
│                     │                   │            │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 128)       │     16,512 │ add_6[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 128)       │          0 │ add_6[0][0],    

 Total params: 184,329 (720.04 KB)

 Trainable params: 184,329 (720.04 KB)

 Non-trainable params: 0 (0.00 B)

### Discriminator
The generator takes as an input the conditions and the target features (either generated or from the reference sample) and provide the probability for the sample of being part of the reference sample.

In formula,
$$
d^{(th)}: \mathbb R^{N_I} \otimes \mathbb R^{N_O} \to [0, 1] \subset \mathbb R.
$$

Usually, to map the response of a neural network in the interval $[0, 1]$, the sigmoid function is used.
At implementation level, we decided to move the evaluation of the sigmoid from the activation of the last layer to the computation of the loss function.
This is believed to improve the numerical stability of the computation by avoiding taking the exponential of a logarithm.

In practice, our implemented discriminator will be described by
$$
d^{(impl)}: \mathbb R^{N_I} \otimes \mathbb R^{N_O} \to \mathbb R.
$$


In terms of implemntation, the tensor we are passing as an input to the neural network for each batch is composed as depicted
in the following table

<table width=300>
    <tr><th colspan=2>$X$<th><th>$y$</tr>
    <tr><td>Input conditions (gen. level features)<td><b>Reference</b> target features<td><td><b>1</b></tr>
    <tr><td>Input conditions (gen. level features)<td><b>Generated</b> target features<td><td><b>0</b></tr>
</table>

The input conditions are repeated twice, but in the first half of the batch 
they are completed with the output features of the reference samples and labeld as $1$.
In the second half of the batch they are completed with randomly generated features and labeld with $0$.


In [5]:
d_input_ref_x = tf.keras.layers.Input(shape=[X.shape[1]])

d_input_ref_y = tf.keras.layers.Input(shape=[y.shape[1]])
d_input_gen_y = tf.keras.layers.Input(shape=[y.shape[1]])

d_input_y = tf.keras.layers.Concatenate(axis=0)((d_input_ref_y, d_input_gen_y))
d_input_x = tf.keras.layers.Concatenate(axis=0)((d_input_ref_x, d_input_ref_x))

d_input = tf.keras.layers.Concatenate(axis=1)((d_input_x, d_input_y))

d_dense_cfg=dict(units=128, activation='tanh', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.L2(1e-3))

x = tf.keras.layers.Dense(**d_dense_cfg)(d_input)
for i in range(10):
    x_ = tf.keras.layers.Dense(**d_dense_cfg)(x)
    x = x + x_

output = tf.keras.layers.Dense(1)(x)

discriminator = tf.keras.Model(inputs=[d_input_ref_x, d_input_ref_y, d_input_gen_y], outputs=[output])
discriminator.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 12)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 12)        │          0 │ input_layer_2[0]… │
│ (Concatenate)       │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 9)         │          0 │ input_layer_3[0]… │
│ (Concatenate)       │                   │            │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 21)        │          0 │ concatenate_2[0]… │
│ (Concatenate)       │                   │            │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 128)       │      2,816 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 128)       │     16,512 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_10 (Add)        │ (None, 128)       │          0 │ dense_12[0][0],   │
│                     │                   │            │ dense_13[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 128)       │     16,512 │ add_10[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_11 (Add)        │ (None, 128)       │          0 │ add_10[0][0],     │
│                     │                   │            │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 128)       │     16,512 │ add_11[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_12 (Add)        │ (None, 128)       │          0 │ add_11[0][0],     │
│                     │                   │            │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 128)       │     16,512 │ add_12[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_13 (Add)        │ (None, 128)       │          0 │ add_12[0][0],     │
│                     │                   │            │ dense_16[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 128)       │     16,512 │ add_13[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_14 (Add)        │ (None, 128)       │          0 │ add_13[0][0],     │
│                     │                   │            │ dense_17[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 128)       │     16,512 │ add_14[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_15 (Add)        │ (None, 128)       │          0 │ add_14[0][0],     │
│                     │                   │            │ dense_18[0][0]  

 Total params: 168,065 (656.50 KB)

 Trainable params: 168,065 (656.50 KB)

 Non-trainable params: 0 (0.00 B)

### Referee network
The referee network mimics the discriminator network, therefore
$$
r: \mathbb R^{N_I} \otimes \mathbb R^{N_O} \to \mathbb R
$$
The input tensor is built in the same way as for the discriminator.

In [6]:
r_input_ref_x = tf.keras.layers.Input(shape=[X.shape[1]])

r_input_ref_y = tf.keras.layers.Input(shape=[y.shape[1]])
r_input_gen_y = tf.keras.layers.Input(shape=[y.shape[1]])

r_input_y = tf.keras.layers.Concatenate(axis=0)((r_input_ref_y, r_input_gen_y))
r_input_x = tf.keras.layers.Concatenate(axis=0)((r_input_ref_x, r_input_ref_x))

r_input = tf.keras.layers.Concatenate(axis=1)((r_input_x, r_input_y))

r_dense_cfg=d_dense_cfg

x = tf.keras.layers.Dense(**r_dense_cfg)(r_input)
for i in range(10):
    x_ = tf.keras.layers.Dense(**r_dense_cfg)(x)
    x = x + x_

output = tf.keras.layers.Dense(1)(x)

referee = tf.keras.Model(inputs=[r_input_ref_x, r_input_ref_y, r_input_gen_y], outputs=[output])
referee.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 12)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 12)        │          0 │ input_layer_5[0]… │
│ (Concatenate)       │                   │            │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 9)         │          0 │ input_layer_6[0]… │
│ (Concatenate)       │                   │            │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 21)        │          0 │ concatenate_5[0]… │
│ (Concatenate)       │                   │            │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 128)       │      2,816 │ concatenate_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_25 (Dense)    │ (None, 128)       │     16,512 │ dense_24[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_20 (Add)        │ (None, 128)       │          0 │ dense_24[0][0],   │
│                     │                   │            │ dense_25[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_26 (Dense)    │ (None, 128)       │     16,512 │ add_20[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_21 (Add)        │ (None, 128)       │          0 │ add_20[0][0],     │
│                     │                   │            │ dense_26[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_27 (Dense)    │ (None, 128)       │     16,512 │ add_21[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_22 (Add)        │ (None, 128)       │          0 │ add_21[0][0],     │
│                     │                   │            │ dense_27[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_28 (Dense)    │ (None, 128)       │     16,512 │ add_22[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_23 (Add)        │ (None, 128)       │          0 │ add_22[0][0],     │
│                     │                   │            │ dense_28[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_29 (Dense)    │ (None, 128)       │     16,512 │ add_23[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_24 (Add)        │ (None, 128)       │          0 │ add_23[0][0],     │
│                     │                   │            │ dense_29[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_30 (Dense)    │ (None, 128)       │     16,512 │ add_24[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_25 (Add)        │ (None, 128)       │          0 │ add_24[0][0],     │
│                     │                   │            │ dense_30[0][0]  

 Total params: 168,065 (656.50 KB)

 Trainable params: 168,065 (656.50 KB)

 Non-trainable params: 0 (0.00 B)

### Training step

The training step is defined with the lower-level tensorflow API because we need to carefully
tune which weights we wish to update based on each evaluation of a loss function.

Technically, we are using the tensorflow [GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) 
to keep track of the gradient while we describe the computation of the loss function.
We will have a different tape for each neural network, recording the derivatives of the 
loss functions with respect to the weights of that particular network.

#### Notes on the chosen loss function
The loss function for the classification task is clearly a Binary Cross Entropy (BCE).
However we adopt two non-default options for its computation:
 * `from_logit=True`, to improve the numerical stability of the gradient computation, which
   is of particular importance for GANs because of the very long training procedure that 
   may inflate the errors due to many subsequent iterations
 * `label_smoothing=0.1`, to introduce a penalty against overconfident classification, which 
   corresponds to the *plateaux* of the sigmoid function, where the gradient is null, providing
   no useful information for the generator's training.


In [21]:
compute_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)
g_optimizer = tf.keras.optimizers.Adam(1e-4)
d_optimizer = tf.keras.optimizers.Adam(1e-4)
r_optimizer = tf.keras.optimizers.Adam(5e-4)

def weights(model):
    return  [w.value for w in model.trainable_weights]

@tf.function
def train_step(dataset):
    # Unpack the dataset to retrieve input and output
    X, y_ref = dataset
    
    # Compute the batch size
    batch_size = tf.shape(X)[0]
    
    # Prepares the tensor of labels for the discriminator, based on the batch_size
    label_true = tf.concat((tf.ones(batch_size), tf.zeros(batch_size)), axis=0)

    # Generate the random noise. Once per batch.
    r = tf.random.normal([batch_size, random.shape[1]], 0., 1.)
    
    # Begins the computation of the loss, tracking the gradient
    with tf.GradientTape(watch_accessed_variables=False) as tape_generator:
        with tf.GradientTape(watch_accessed_variables=False) as tape_discriminator:
            # The gradient will be computed with respect to the variables defined here
            tape_generator.watch(weights(generator))
            tape_discriminator.watch(weights(discriminator))
            
            # Run the generator
            y_gen = generator((X, r), training=True)

            # Run the discriminator and obtain the logits
            label_pred = discriminator((X, y_ref, y_gen), training=True)
            
            # Convert the logits into a loss function
            d_loss = compute_loss(label_true, label_pred)
            
            # Invert the loss function of the discriminator to obtain the loss 
            # of the generator
            g_loss = -d_loss
            
    # Begins the computation of the loss of the referee network
    # Note that it is completely indepenent of the generator training
    with tf.GradientTape() as tape_referee:
        # Defines the variables
        #tape_referee.watch(weights(referee))
        # Runs the referee discriminator
        label_referee = referee((X, y_ref, y_gen))
        # Compute the loss of the referee network
        r_loss = compute_loss(label_true, label_referee)

    # Compute the gradients of the loss of each NN wrt. to its weights
    g_vars = weights(generator)
    d_vars = weights(discriminator)
    r_vars = weights(referee)
    
    d_grads = tape_discriminator.gradient(d_loss, d_vars)
    g_grads = tape_generator.gradient(g_loss, g_vars)
    r_grads = tape_referee.gradient(r_loss, r_vars)
    
    ## Apply the gradients
    g_optimizer.apply_gradients(zip(g_grads, g_vars))
    d_optimizer.apply_gradients(zip(d_grads, d_vars))
    r_optimizer.apply_gradients(zip(r_grads, r_vars))
    
    ## Return the value of the loss for monitoring purpose
    return r_loss

    
@tf.function
def test_step(dataset):
    X, y_ref = dataset
    batch_size = tf.shape(X)[0]
    r = tf.random.normal([batch_size, random.shape[1]], 0., 1.)
    
    y_gen = generator((X, r))

    label_pred = referee((X, y_ref, y_gen))
    label_true = tf.concat((tf.ones(batch_size), tf.zeros(batch_size)), axis=0)

    return compute_loss(label_true, label_pred)
    

## Training

 * Batch size: 1k
 * Number of epochs: 1000

In [22]:
from tqdm import trange, tqdm

losses = []
validation_loss = []
TRAINING_TIME_LIMIT_SECONDS = int(environ.get("PRE_TRAINING_TIME_LIMIT_SECONDS", "7200"))

vdst = next(iter(validation_dataset.batch(100_000)))

progress_bar = trange(1000)

start_time = time.time()
for epoch in progress_bar:
    epoch_losses = []
    for dset in train_dataset.batch(1000):
        epoch_losses.append(train_step(dset))
    losses.append(np.mean(epoch_losses))
    validation_loss.append(test_step(vdst))
    if time.time() - start_time > TRAINING_TIME_LIMIT_SECONDS:
        progress_bar.set_description(
            f"Time limit reached. Training (validation): "
            f"{losses[-1]:.3f} ({validation_loss[-1]:.3f})"
        )
        break
   
    
    progress_bar.set_description(f"Training (validation) loss: {losses[-1]:.3f} ({validation_loss[-1]:.3f})")


  0%|          | 0/1000 [01:34<?, ?it/s]

Time limit reached. Training (validation): 0.233 (0.226):   0%|          | 0/1000 [00:21<?, ?it/s]


The evolution of the loss function as evaluated by the referee network is reported below.
A dashed line represent the ideal value of the BCE when evaluated on two identical datasets with an ideal classifier.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses, label="Training data")
plt.plot(validation_loss, label="Validation data")
plt.plot([0, len(losses)], [np.log(2)]*2, 'k--')

plt.xlabel("Epoch")


plt.ylabel("Binary crossentropy")
plt.legend()
plt.show()

## Exporting the model
The model is exported to the same directory were the preprocessing steps `tX` and `tY` were stored.

In [23]:
import os
default_output_model = "/tmp/s3models/lamarr-train/models/resolution/model.keras"
output_model = environ.get('OUTPUT_MODEL', default_output_model)
base_dir = os.path.dirname(output_model)
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
generator.save(output_model)

## Conclusion
In this notebook we discussed the training procedure of the GAN model used to parametrize the resolution.

In particular, we discussed
 * the overall structure of the DNN system;
 * the architecture of the generator, discriminator and of a referee network we introduced to ease monitoring, debugging an hyperparameter optimization
 * the procedure for optimizing the weights of the three networks based on three different computations of the gradients
 * the outcome of the training procedure as visualized by the evolution of the loss of the referee network

Finally, we exported the model for deployment and further validation.
